# *Tradução de PDF*

In [ ]:
Estou testando algumas bibliotecas para traduzir um PDF dos meus estudos de Espanhol. Pretendo testar com inglês também. Uso de pdf2image e googletrans

In [1]:
pip install pdf2image


In [2]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 996.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=515a8e12c56d5b43e1c6117fa20c37c6b2a81272e345215858cac6a6fab56ff8
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0

In [6]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.9 MB/s eta 0:00:00


In [3]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (485 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [14]:
pip install --upgrade PyPDF2

In [17]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            # Apenas extrai o texto da página, sem argumentos adicionais
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Dividindo o texto em partes menores para evitar sobrecarga da API de tradução
        translator = Translator()
        text_parts = [text[i:i + 5000] for i in range(0, len(text), 5000)]
        translated_text = ""

        for part in text_parts:
            # Traduzir o texto extraído em partes menores
            translated_text += translator.translate(part, dest='pt').text

        # Exibir a tradução
        display(HTML(f"<h3>Texto traduzido:</h3><pre>{translated_text}</pre>"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [18]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos, subtítulos e listas.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos, subtítulos e listas.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)
    # Expressão regular para listas de itens (sequências de palavras com tabs ou múltiplos espaços)
    list_pattern = re.compile(r'([a-záéíóúñ]+(\t|\s{2,})[a-záéíóúñ]+)+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    # Substituindo as listas por uma formatação de lista em HTML
    formatted_text = re.sub(list_pattern, lambda match: f'<ul><li>' + '</li><li>'.join(match.group(0).split()) + '</li></ul>', formatted_text)

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e listas
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Exibir a tradução formatada
        display(HTML(f"<h3>Texto traduzido:</h3><pre>{translated_text}</pre>"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [19]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos, removendo listas.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    # Adicionando quebras de parágrafo
    formatted_text = re.sub(r'(\n\s*\n)', r'</p><p>', formatted_text)
    formatted_text = f'<p>{formatted_text.strip()}</p>'

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e subtítulos
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Exibir o texto original com fundo cinza e negrito
        original_text_html = f'<div style="background-color: #f0f0f0; font-weight: bold;">{formatted_text}</div>'

        # Exibir a tradução formatada
        display(HTML(f"<h3>Texto original:</h3>{original_text_html}<h3>Texto traduzido:</h3><pre>{translated_text}</pre>"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [20]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos, removendo listas.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    # Adicionando quebras de parágrafo
    formatted_text = re.sub(r'(\n\s*\n)', r'</p><p>', formatted_text)
    formatted_text = f'<p>{formatted_text.strip()}</p>'

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e subtítulos
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Intercalar texto original e traduzido
        intercalated_text = ""
        formatted_text_parts = formatted_text.split('</p><p>')
        translated_text_parts = translated_text.split('</p><p>')

        for original, translated in zip(formatted_text_parts, translated_text_parts):
            intercalated_text += f'<div style="background-color: #f0f0f0; font-weight: bold; padding: 5px;">{original}</div>'
            intercalated_text += f'<div style="padding: 5px;">{translated}</div>'

        if len(formatted_text_parts) > len(translated_text_parts):
            remaining_original = ''.join(formatted_text_parts[len(translated_text_parts):])
            intercalated_text += f'<div style="background-color: #f0f0f0; font-weight: bold; padding: 5px;">{remaining_original}</div>'

        # Exibir o texto intercalado
        display(HTML(f"<h3>Texto original e traduzido:</h3>{intercalated_text}"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')

In [21]:
from PyPDF2 import PdfReader
from googletrans import Translator
from ipywidgets import FileUpload
from IPython.display import display, HTML
import io
import re
import logging

# Configuração do logging para depuração
logging.basicConfig(filename='app.log', level=logging.DEBUG)

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens.

    Args:
        pdf_bytes: Conteúdo binário do PDF.

    Returns:
        str: Texto extraído ou None em caso de erro.
    """
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        logging.exception("Erro ao extrair texto do PDF")
        return None

def format_text(text):
    """Formata o texto extraído para identificar títulos e subtítulos, removendo listas.

    Args:
        text: O texto extraído do PDF.

    Returns:
        str: Texto formatado com detecção de títulos e subtítulos.
    """
    # Expressão regular para identificar títulos principais (1., 2., 3., etc.)
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    # Expressão regular para identificar subtítulos (1.1, 2.1, 2.2, etc.)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    # Substituindo os títulos principais para adicionar um cabeçalho em HTML
    formatted_text = re.sub(title_pattern, r'<h1>\g<0></h1>', text)

    # Substituindo os subtítulos para adicionar um subtítulo em HTML
    formatted_text = re.sub(subtitle_pattern, r'<h2>\g<0></h2>', formatted_text)

    # Adicionando quebras de parágrafo
    formatted_text = re.sub(r'(\n\s*\n)', r'</p><p>', formatted_text)
    formatted_text = f'<p>{formatted_text.strip()}</p>'

    return formatted_text

def handle_pdf_upload(change):
    """Processa o upload do PDF.

    Args:
        change: Evento de mudança do widget FileUpload.
    """
    try:
        if not change.new:
            display(HTML("<h3>Erro:</h3><pre>Nenhum arquivo foi carregado.</pre>"))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("<h3>Erro:</h3><pre>O conteúdo do arquivo está vazio ou não foi carregado corretamente.</pre>"))
            return

        text = extract_text_from_pdf(content)

        if not text:
            display(HTML("<h3>Erro:</h3><pre>Não foi encontrado texto no PDF ou houve um problema na extração.</pre>"))
            return

        # Verificar se o texto extraído não está vazio
        if len(text.strip()) == 0:
            display(HTML("<h3>Erro:</h3><pre>O texto extraído está vazio.</pre>"))
            return

        # Formatar o texto para identificar títulos e subtítulos
        formatted_text = format_text(text)

        # Traduzir o texto formatado
        translator = Translator()
        text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
        translated_text = ""

        for part in text_parts:
            translated_text += translator.translate(part, dest='pt').text

        # Intercalar texto original e traduzido
        intercalated_text = ""
        formatted_text_parts = formatted_text.split('</p><p>')
        translated_text_parts = translated_text.split('</p><p>')

        for original, translated in zip(formatted_text_parts, translated_text_parts):
            intercalated_text += (
                f'<div style="background-color: #f0f0f0; font-weight: bold; padding: 10px;">{original}</div>'
                f'<div style="padding: 10px;">{translated}</div>'
            )

        if len(formatted_text_parts) > len(translated_text_parts):
            remaining_original = ''.join(formatted_text_parts[len(translated_text_parts):])
            intercalated_text += (
                f'<div style="background-color: #f0f0f0; font-weight: bold; padding: 10px;">{remaining_original}</div>'
            )

        # Exibir o texto intercalado
        display(HTML(f"<h3>Texto original e traduzido:</h3>{intercalated_text}"))
    except Exception as e:
        logging.exception("Erro ao processar o arquivo")
        display(HTML(f"<h3>Erro ao processar o arquivo:</h3><pre>{str(e)}</pre>"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)


FileUpload(value={}, accept='.pdf', description='Upload')